In [ ]:
import pandas as pd
from datetime import datetime, timedelta
csv_file = '../Data_Cleanup/outCSV/Metro_Set_with_IdleTime.csv'
metro = pd.read_csv(csv_file)

In [ ]:
metro['start_time'] = pd.to_datetime(metro['start_time'])

metro ['end_time'] = pd.to_datetime(metro['end_time'])

metro = metro.sort_values(by='start_time')

In [ ]:
#Create the corresponding columns
metro["morgens"] = 0
metro["vormittags"] = 0
metro["mittags"] = 0
metro["nachmittags"]=0
metro["abends"]=0
metro["nachts"]=0

#set the start and end of each day time
morgens = range(5,9)
vormittags = range(9,12)
mittags = range(12,15)
nachmittags = range(15,19)
abends = range(19, 23)
nachts = [23,0,1,2,3,4]


In [ ]:
#Make sure to change the "idle_time" to a timedelta object, so we can use .totalseconds() later
metro["idle_time"] = pd.to_timedelta(metro["idle_time"])

In [ ]:
#This list will serve as string provider that we can insert later, to access the proper column
TagesZeiten = ["morgens", "vormittags", "mittags", "nachmittags", "abends", "nachts"]
#This list will serve to provide the start time of each day time, so we know when to change columns
Intervall_starts = [5,9,12,15,19,23]


# Iterate through each row in the DataFrame
for i, row in metro.iterrows():
    #we only need the starting time hour
    start_hour = row["start_time"].hour
    # and we need to know the duration (idle time) we need to look back
    idle_time = row["idle_time"]
    # make sure to skip any rows with no idle times, since they have 0s in the day times columns
    if row["idle_time"] is None:
        continue
    try:
        #calculate duration
        dauer = float(idle_time.total_seconds() / 3600)
        #assign the corresponding index depending on the time. This index will allow us to access different day times columns and compare start times
        if start_hour in morgens:
            index = 0
        elif start_hour in vormittags:
            index = 1
        elif start_hour in mittags:
            index = 2
        elif start_hour in nachmittags:
            index = 3
        elif start_hour in abends:
            index = 4
        elif start_hour in nachts:
            index = 5
        # while we have idle time and we haven't gone through every columns already we can loop through daytimes backwards and fill with 1 
        while dauer >0 and index > -6:
            metro.at[i, TagesZeiten[index]] = 1
            temp = abs(Intervall_starts[index]- start_hour)

            dauer -= temp
            start_hour -= temp
            index -= 1
    #handle the NaN values since we changed metro["idle_time"] to a timedelta.
    except ValueError:
        continue



In [ ]:
#Export
metro.to_csv('../Data_Cleanup/outCSV/Metro_Set_DayTimes.csv')